<a href="https://colab.research.google.com/github/jasourobeidat/DetailedProjectArabia/blob/master/SAIA_Custom_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install bs4

In [0]:
from bs4 import BeautifulSoup
import requests

Collect list of Images

In [0]:
base_url = "https://www.oldbookillustrations.com/subjects/people/page/"
#base_url = "https://www.oldbookillustrations.com/subjects/people/animals/"
page_number = 1

max_images = 400
image_pages = []

complete = False
while(not complete):

  page_url = "{}{}/".format(base_url, str(page_number))
  r  = requests.get(page_url)

  if r.status_code != 200:
    print("{} : Error retrieveing {}".format(r.status_code, page_url))
    break

  soup = BeautifulSoup(r.text)

  figure_elements = soup.find_all("figure")
  for figure_element in figure_elements:
    if(len(image_pages) < max_images):
      image_pages.append(figure_element.a['href'])
    else:
      complete = True
      break

  page_number = page_number +1

print("{} images found".format(len(image_pages)))

Download and Save

In [0]:
from PIL import Image
from io  import BytesIO
import os

OUTPUT_PATH = "Images"
os.makedirs(OUTPUT_PATH, exist_ok = True)


for i, img_page in enumerate(image_pages):
  r  = requests.get(img_page)
  if r.status_code != 200:
    print("{} : Error retrieveing {}".format(r.status_code, page_url))

  soup = BeautifulSoup(r.text)
  dl_list = soup.find('p', id='highres-dld').find_all('a')
  for dl in dl_list:
    if(dl.contents[0] == "768 px"):
      url = dl["href"]
      image_url = "https://www.oldbookillustrations.com" + url
      response = requests.get(image_url)
      img = Image.open(BytesIO(response.content))
      img_path = "image_{:04d}.jpg".format(i)
      img.save(os.path.join(OUTPUT_PATH, img_path))

      print("Saving... {}".format(img_path))
      break


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!zip -r images.zip /content/Images

In [0]:
!mkdir -p "/content/gdrive/My Drive/Other Projects/SAIA Course/Scraped_images"
!cp -rf /content/images.zip "/content/gdrive/My Drive/Other Projects/SAIA Course/Scraped_images"